# MAX-CUT for graphs

In [1]:
import numpy as np
import scipy as sp
import math
import pulp
from random import *
from pyvis.network import Network as netviz
import networkx as net

### Generate a G(n,p)-random graph

In [91]:
N,p = 100, 0.05
G = net.gnp_random_graph(N,p)
print(len(G.nodes))
print(len(G.edges))

100
228


## Create the adjacency matrix of G
The function `adjacency_matrix()` returns a sparse matrix (i.e. only the coordinates of non-null entries). We turn it into a dense matrix before we can use it with the function `todense()`.

In [3]:
A = net.adjacency_matrix(G).todense()

## Define a function to calculate the cut number for given `x`

In [85]:
def CUT(G,x):
    A = net.adjacency_matrix(G).todense()
    n = len(G.nodes)
    cut = 0
    if len(x) != n:
        print("ERROR length of x does not match dimension of G")
        return(cut)
    else:
        for i in range(1,n):
            for j in range(1,n):
                cut += 1/4*A.A[i][j]*(1-x[i]*x[j])
        return(cut)  

## Generate $X \sim \mathcal{U} (\{-1,1\})$ and calculate CUT(G,X)

In [86]:
def rand_part(G):
    "return a random partition of V(G) represented by a vector with independent symmetric Bernoulli entries"
    X = []
    for i in range(N):
        X.append(1-2*round(random()))
    return(X)
X = rand_part(G)
CUT(G,X)

121.0

## Define a (brute-force) function to calculate MAXCUT
*WARNING* the runtime is $O(2^n)$ and _will_ take long for Graph sizes above $N=15$

In [87]:
def iterate(x):
    x[0] = x[0]*(-1)
    for i in range(len(x)):
        if (x[i] == 1 ):
            try:
                x[i+1] = x[i+1]*(-1)
            except IndexError:
                break
        else:
            break
    return(x)
    
    
def MAXCUT(G):
    n = len(G.nodes)
    x = [1 for i in range(n)]
    x0 = x
    cut = 0
    h = 0
    for i in range (2**n):
        h = CUT(G,x)
        if h > cut:
            cut = h
            x0 = x
            print(f'found a better CUT. NEW CUT# = {cut}')
        x = iterate(x)
    return(cut,x0)
        
    
    

## A $(0.5-\varepsilon)$-approximation algorithm for MAXCUT
Let $\varepsilon>0$ we start with a random partition of $V$, represented by a vector $X$ with independent symmetric Bernoulli entries. Then flip the entries of $X$ one by one and check if the cut number of the new cutset `CUT(G,X)` has improved. If so, update $X$ and continue until `CUT(G,X)` $\geq (0.5-\varepsilon)|E|$. As long as there are vertices where less than half of the incident edges are cut, the cut number is improved by flipping such a vertex. Hence, the algorithm improves can find a vertex to flip as long as `CUT(G,X)` $< 0.5 |E|$. Furthermore, the cut number increases by at least 1 with every flip, so the the number of flips is upper bounded by $|E|$.

**Exercise** Implement this algorithm!

In [92]:
# Stefans solution

def find_lower_edges(G,v):
    """helper function for maxcut_approx_Stefan"""
    r = []
    for u in G.neighbors(v):
        if u < v:
            r.append(u)
    return(r)

def v1_or_v2(G,V1,v):
    """helper function for maxcut_approx_Stefan"""
    ln = find_lower_edges(G,v)
    cnt = 0
    r = False
    for u in ln:
        if u in V1:
            cnt += 1
    if cnt <= len(ln)/2:
        r = True
    return(r)

def maxcut_approx_Stefan(G):
    N = len(G.nodes())
    V1 = [0]
    for v in range(1,N):
        if v1_or_v2(G,V1,v):
            V1.append(v)
    X = [-1 for i in range(N)]
    print(V1)
    for v in V1:
        X[v] = 1
    return(X)
    
X = maxcut_approx_Stefan(G)

print(CUT(G,X))


[0, 1, 2, 3, 4, 6, 7, 9, 11, 12, 15, 16, 18, 19, 21, 22, 26, 27, 29, 31, 32, 33, 34, 35, 36, 38, 40, 42, 46, 47, 48, 52, 54, 55, 56, 58, 62, 63, 67, 69, 71, 72, 73, 77, 78, 79, 80, 81, 87, 89, 91, 96, 99]
164.0


In [93]:
# Thomas' solution
def maxcut_approx_Thomas(G):
    X = rand_part(G)
    Cutnumber = CUT(G, X)
    counter = 0 
    while counter < N:
        for i in range(N):
            X[i] = -X[i]
            if CUT(G, X) > Cutnumber:
                Cutnumber = CUT(G, X)
                counter = 0 
            else:
                X[i] = -X[i]
                counter = counter + 1
    return(X)

CUT(G, maxcut_approx_Thomas(G)) 

175.0

## Draw the partitioned Graph

In [94]:
def draw_part_graph(G,x):
    n = len(G.nodes)
    if len(x) != n:
        print("ERROR length of x does not match dimension of G")
        return
    for v in G.nodes():
        if x[v] == 1:
            G.nodes[v]["color"] = "red"
        else:
            G.nodes[v]["color"] = "blue"
    edges = [e for e in G.edges]
    for e in edges:
        if x[e[0]]*x[e[1]] == -1:
            G.edges[e]["color"] = "red"
        else:
            G.edges[e]["color"] = "black"
    viz = netviz(height=800, width=800, notebook=True)
    #viz.barnes_hut()
    #viz.repulsion()
    #viz.hrepulsion()
    viz.force_atlas_2based()
    viz.from_nx(G)
    viz.show("net.html")    
    return(viz)

viz = draw_part_graph(G,X)
viz.show("net.html")


## Calculate the MAXCUT-norm of G

In [12]:
cut, x0 = MAXCUT(G)
print(cut)

found a better CUT. NEW CUT# = 6.0
found a better CUT. NEW CUT# = 8.0
found a better CUT. NEW CUT# = 11.0
found a better CUT. NEW CUT# = 13.0
found a better CUT. NEW CUT# = 14.0
found a better CUT. NEW CUT# = 17.0
found a better CUT. NEW CUT# = 19.0
found a better CUT. NEW CUT# = 22.0
found a better CUT. NEW CUT# = 24.0
found a better CUT. NEW CUT# = 27.0
found a better CUT. NEW CUT# = 29.0
found a better CUT. NEW CUT# = 30.0
found a better CUT. NEW CUT# = 32.0
found a better CUT. NEW CUT# = 35.0
found a better CUT. NEW CUT# = 37.0
found a better CUT. NEW CUT# = 40.0
found a better CUT. NEW CUT# = 42.0
found a better CUT. NEW CUT# = 44.0
found a better CUT. NEW CUT# = 47.0
found a better CUT. NEW CUT# = 49.0
found a better CUT. NEW CUT# = 50.0
found a better CUT. NEW CUT# = 52.0
found a better CUT. NEW CUT# = 54.0
found a better CUT. NEW CUT# = 55.0
found a better CUT. NEW CUT# = 57.0
found a better CUT. NEW CUT# = 59.0
found a better CUT. NEW CUT# = 60.0
found a better CUT. NEW CUT# =

KeyboardInterrupt: 